# Stworzenie SparkSession

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.sql import Window

spark = SparkSession.builder \
    .appName('Mieszkania_ML') \
    .master('local[*]') \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1622890252275_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType
from pyspark.sql.window import Window
from pyspark.ml import Pipeline

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Zaczytanie danych

In [4]:
df = spark.read.json("s3://mieszkania2-pl/db/*.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = df.withColumn("Wojewodztwo", \
                   f.split(f.element_at(f.split(f.element_at(f.split(f.input_file_name(), "/"), -1), "_"), -1), "\.")[0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = df.withColumn("Miejscowosc", f.initcap(f.split("Lokalizacja", ", ")[0]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df = df.distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Zmiana nazw kolumn

In [8]:
df = df.withColumnRenamed('Liczba pokoi ', 'Liczba pokoi')\
.withColumnRenamed('Liczba pi\u0119ter w budynku ', 'Liczba pieter w budynku')\
.withColumnRenamed('Materia\u0142 ', 'Material')\
.withColumnRenamed('Pi\u0119tro ', 'Pietro')\
.withColumnRenamed('Rok budowy ', 'Rok budowy')\
.withColumnRenamed('Szeroko\u015b\u0107 geograficzna', 'Szerokosc geograficzna')\
.withColumnRenamed('D\u0142ugo\u015b\u0107 geograficzna', 'Dlugosc geograficzna')\
.withColumnRenamed('Rok budowy ', 'Rok budowy')\
.withColumnRenamed('Powierzchnia ca\u0142kowita', 'Powierzchnia calkowita')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Wybór kolumn

In [9]:
df = df.select("Cena", "Cena za m2", "Szerokosc geograficzna", "Dlugosc geograficzna",\
               "Kod pocztowy", "Miejscowosc", "Wojewodztwo",\
               "Powierzchnia calkowita", "Liczba pokoi", "Pietro", "Liczba pieter w budynku",\
               "Rok budowy", "Material", "Typ budynku")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Cena: string (nullable = true)
 |-- Cena za m2: string (nullable = true)
 |-- Szerokosc geograficzna: double (nullable = true)
 |-- Dlugosc geograficzna: double (nullable = true)
 |-- Kod pocztowy: string (nullable = true)
 |-- Miejscowosc: string (nullable = true)
 |-- Wojewodztwo: string (nullable = true)
 |-- Powierzchnia calkowita: string (nullable = true)
 |-- Liczba pokoi: string (nullable = true)
 |-- Pietro: string (nullable = true)
 |-- Liczba pieter w budynku: string (nullable = true)
 |-- Rok budowy: string (nullable = true)
 |-- Material: string (nullable = true)
 |-- Typ budynku: string (nullable = true)

# Preprocessing

##### Potrzebne funkcje

In [11]:
def find_between(s, first, last):
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except ValueError:
        return ""

def num_filter(a_string):
    try:
        if a_string.find("EUR") == -1:
            if a_string.find(",") == -1:
                return "".join(filter(str.isdigit, a_string))
            else:
                s1 = find_between(a_string, "", ",")
                s2 = find_between(a_string, ",", " ")
                return "".join(filter(str.isdigit, s1)) + "." + s2
        else:
            a_string = a_string[a_string.find("EUR"):]
            if a_string.find(",") == -1:
                return "".join(filter(str.isdigit, a_string))
            else:
                s1 = find_between(a_string, "", ",")
                s2 = find_between(a_string, ",", " ")
                return "".join(filter(str.isdigit, s1)) + "." + s2
    except:
        pass
    
udf_num_filter = f.udf(num_filter, StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Usuwanie spacji, przecinków itp. z wartości numerycznych

In [12]:
df = df.withColumn('Cena', udf_num_filter(f.col("Cena")))\
.withColumn('Cena za m2', f.regexp_replace('Cena za m2', ' zł/m2', ''))\
.withColumn('Cena za m2', f.regexp_replace('Cena za m2', '\u00a0', ''))\
.withColumn('Cena za m2', f.regexp_replace('Cena za m2', ',', '.'))\
.withColumn('Powierzchnia calkowita', f.regexp_replace('Powierzchnia calkowita', ' m2', ''))\
.withColumn('Powierzchnia calkowita', f.regexp_replace('Powierzchnia calkowita', ',', '.'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Konwercja typów danych numerycznych (string -> float)

Najpierw zmiana "Parter" na wartość 0, aby przekonwertować do float

In [13]:
df = df.withColumn("Pietro", f.when(df["Pietro"] == "Parter", 0).otherwise(df["Pietro"]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Konwersja

In [14]:
df = df.withColumn('Cena', df['Cena'].cast(FloatType()))\
.withColumn('Cena za m2', df['Cena za m2'].cast(FloatType()))\
.withColumn('Liczba pokoi', df['Liczba pokoi'].cast(FloatType()))\
.withColumn('Liczba pieter w budynku', df['Liczba pieter w budynku'].cast(FloatType()))\
.withColumn('Pietro', df['Pietro'].cast(FloatType()))\
.withColumn('Powierzchnia calkowita', df['Powierzchnia calkowita'].cast(FloatType()))\
.withColumn('Rok budowy', df['Rok budowy'].cast(FloatType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Czyszczenie danych

##### Funkcje czyszczące outliery ręcznie

In [15]:
def clean_cena(df, low=10000, high=20000000, dropna=False):
    if dropna == True:
        return df.filter((df["Cena"] >= low) & (df["Cena"] <= high))
    elif dropna == False:
        return df.filter(((df["Cena"] >= low) & (df["Cena"] <= high)) | df["Cena"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def clean_cena_za_m2(df, low=1000, high=50000, count_price_per_m2=True, dropna=False):
    if count_price_per_m2 == True:
        df = df.withColumn("Cena za m2", df["Cena"]/df["Powierzchnia calkowita"])
    if dropna == True:
        return df.filter((df["Cena za m2"] >= low) & (df["Cena za m2"] <= high))
    elif dropna == False:
        return df.filter(((df["Cena za m2"] >= low) & (df["Cena za m2"] <= high)) | df["Cena za m2"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def clean_lat_lon(df, lat_min=49, lat_max=(54 + 50/60), lon_min=(14 + 7/60), lon_max=(24 + 9/60), dropna=False):
    if dropna == True:
        return df.filter((df["Szerokosc geograficzna"] >= lat_min) & \
                         (df["Szerokosc geograficzna"] <= lat_max) & \
                         (df["Dlugosc geograficzna"] >= lon_min) & \
                         (df["Dlugosc geograficzna"] <= lon_max))
    elif dropna == False:
        return df.filter(((df["Szerokosc geograficzna"] >= lat_min) & \
                         (df["Szerokosc geograficzna"] <= lat_max) & \
                         (df["Dlugosc geograficzna"] >= lon_min) & \
                         (df["Dlugosc geograficzna"] <= lon_max)) | \
                         df["Szerokosc geograficzna"].isNull() | \
                         df["Dlugosc geograficzna"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def clean_pietra(df, min=0, max=1000, drop_pietro_greater_than_max=True, dropna=False):
    if dropna == True:
        if drop_pietro_greater_than_max == True:
            return df.filter((df["Pietro"] <= df["Liczba pieter w budynku"]) & \
                             (df["Pietro"] >= min) & \
                             (df["Pietro"] <= max))
        elif drop_pietro_greater_than_max == False:
            return df.filter((df["Pietro"] >= min) & \
                             (df["Pietro"] <= max))
    elif dropna == False:
        if drop_pietro_greater_than_max == True:
            return df.filter(((df["Pietro"] <= df["Liczba pieter w budynku"]) & \
                             (df["Pietro"] >= min) & \
                             (df["Pietro"] <= max)) | \
                             df["Pietro"].isNull() | \
                             df["Liczba pieter w budynku"].isNull())
        elif drop_pietro_greater_than_max == False:
            return df.filter(((df["Pietro"] >= min) & \
                             (df["Pietro"] <= max)) | \
                             df["Pietro"].isNull() | \
                             df["Liczba pieter w budynku"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def clean_pokoje(df, min=1, max=10, dropna=False):
    if dropna == True:
        return df.filter((df["Liczba pokoi"] >= min) & \
                         (df["Liczba pokoi"] <= max))
    elif dropna == False:
        return df.filter(((df["Liczba pokoi"] >= min) & \
                         (df["Liczba pokoi"] <= max)) | \
                         df["Liczba pokoi"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def clean_powierzchnia(df, min=20, max=200, dropna=False):
    if dropna == True:
        return df.filter((df["Powierzchnia calkowita"] >= min) & \
                         (df["Powierzchnia calkowita"] <= max))
    elif dropna == False:
        return df.filter(((df["Powierzchnia calkowita"] >= min) & \
                         (df["Powierzchnia calkowita"] <= max)) | \
                         df["Powierzchnia calkowita"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
def clean_rok_budowy(df, min=1900, max=2024, dropna=False):
    if dropna == True:
        return df.filter((df["Rok budowy"] >= min) & \
                         (df["Rok budowy"] <= max))
    elif dropna == False:
        return df.filter(((df["Rok budowy"] >= min) & \
                         (df["Rok budowy"] <= max)) | \
                         df["Rok budowy"].isNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Usunięcie outlierów

In [22]:
df = clean_cena(df, dropna=True)
df = clean_cena_za_m2(df, dropna=True)
df = clean_lat_lon(df)
df = clean_pietra(df)
df = clean_pokoje(df)
df = clean_powierzchnia(df)
df = clean_rok_budowy(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Przegląd danych

In [23]:
df.show(2, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-------------------------------------
 Cena                    | 603000.0           
 Cena za m2              | 10220.338983050848 
 Szerokosc geograficzna  | 54.36143405        
 Dlugosc geograficzna    | 18.62821847557274  
 Kod pocztowy            | 80-208             
 Miejscowosc             | Gdańsk             
 Wojewodztwo             | pomorskie          
 Powierzchnia calkowita  | 59.0               
 Liczba pokoi            | 3.0                
 Pietro                  | 1.0                
 Liczba pieter w budynku | 3.0                
 Rok budowy              | 2012.0             
 Material                | ytong              
 Typ budynku             | blok               
-RECORD 1-------------------------------------
 Cena                    | 1030514.0          
 Cena za m2              | 14991.475589543888 
 Szerokosc geograficzna  | 54.4126118         
 Dlugosc geograficzna    | 18.6026846         
 Kod pocztowy            | 80-365             
 Miejscowosc 

In [24]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Cena: float (nullable = true)
 |-- Cena za m2: double (nullable = true)
 |-- Szerokosc geograficzna: double (nullable = true)
 |-- Dlugosc geograficzna: double (nullable = true)
 |-- Kod pocztowy: string (nullable = true)
 |-- Miejscowosc: string (nullable = true)
 |-- Wojewodztwo: string (nullable = true)
 |-- Powierzchnia calkowita: float (nullable = true)
 |-- Liczba pokoi: float (nullable = true)
 |-- Pietro: float (nullable = true)
 |-- Liczba pieter w budynku: float (nullable = true)
 |-- Rok budowy: float (nullable = true)
 |-- Material: string (nullable = true)
 |-- Typ budynku: string (nullable = true)

In [25]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

98882

In [26]:
df.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Cena: float, Cena za m2: double, Szerokosc geograficzna: double, Dlugosc geograficzna: double, Kod pocztowy: string, Miejscowosc: string, Wojewodztwo: string, Powierzchnia calkowita: float, Liczba pokoi: float, Pietro: float, Liczba pieter w budynku: float, Rok budowy: float, Material: string, Typ budynku: string]

# ML

### Podział na train, test

In [27]:
train, test = df.randomSplit([0.8, 0.2], seed=10)
train.cache()
test.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Cena: float, Cena za m2: double, Szerokosc geograficzna: double, Dlugosc geograficzna: double, Kod pocztowy: string, Miejscowosc: string, Wojewodztwo: string, Powierzchnia calkowita: float, Liczba pokoi: int, Pietro: int, Liczba pieter w budynku: int, Rok budowy: int, Material: string, Typ budynku: string]

In [28]:
print(f"Zbiór train: {train.count()} rekordów")
print(f"Zbiór test: {test.count()} rekordów")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Zbi?r train: 79235 rekord?w
Zbi?r test: 19647 rekord?w

### Przygotowanie train, test do modelu ML

In [28]:
model_cols = ["Cena za m2", "Kod pocztowy", "Powierzchnia calkowita", "Typ budynku", "Material"]

train = train.select(model_cols) #.withColumn("Kod pocztowy", df["Kod pocztowy"].substr(0,5))
test = test.select(model_cols) #.withColumn("Kod pocztowy", df["Kod pocztowy"].substr(0,5))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
catCols = [x for (x, dataType) in train.dtypes if dataType == 'string']
numCols = [x for (x, dataType) in train.dtypes if ((dataType != 'string') & (x != "Cena za m2"))]
print(f"Zmienne numeryczne: {numCols}")
print(f"Zmienne kategoryczne: {catCols}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Zmienne numeryczne: ['Powierzchnia calkowita']
Zmienne kategoryczne: ['Kod pocztowy', 'Typ budynku', 'Material']

In [30]:
train = train.fillna("other", subset=catCols)
test = test.fillna("other", subset=catCols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### string indexer + one hot encoder + vector assembler + (standard scaler)

In [31]:
from pyspark.ml.feature import Imputer, OneHotEncoderEstimator, StringIndexer, VectorAssembler, StandardScaler

imputer = Imputer(inputCols=numCols, outputCols=numCols)

string_indexer = [
    StringIndexer(inputCol=x, outputCol=x + "_StringIndexer", handleInvalid='skip')
    for x in catCols
]

one_hot_encoder = [
    OneHotEncoderEstimator(
        inputCols=[f"{x}_StringIndexer" for x in catCols],
        outputCols=[f"{x}_OneHotEncoder" for x in catCols],
    )
]

assemblerInput = [x for x in numCols]
assemblerInput += [f"{x}_OneHotEncoder" for x in catCols]

vector_assembler = VectorAssembler(
    inputCols=assemblerInput, outputCol="features_2"
)

scaler = StandardScaler(inputCol="features_2", outputCol="features",
                        withStd=True, withMean=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### pipeline

In [32]:
stages = []
stages += [imputer]
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]
stages += [scaler]

pipeline = Pipeline(stages=stages)

model = pipeline.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_train = model.transform(train).select("Cena za m2", "features")
df_test = model.transform(test).select("Cena za m2", "features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Linear Regression

In [44]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(featuresCol='features', labelCol='Cena za m2', maxIter=10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
paramGrid = ParamGridBuilder()\
    .addGrid(model.regParam, [0.3, 0.1, 0.01])\
    .addGrid(model.fitIntercept, [False, True])\
    .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

crossval = CrossValidator(estimator=model,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(predictionCol="prediction", labelCol="Cena za m2",metricName="rmse"),
                          numFolds=5)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(df_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
cv_predictions = cvModel.transform(df_train)
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE train: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE train: 2149.8048223770375

In [48]:
cv_predictions = cvModel.transform(df_test)
cv_predictions.select("prediction","Cena za m2","features").show(10)

from pyspark.ml.evaluation import RegressionEvaluator
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE test: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|        prediction|        Cena za m2|            features|
+------------------+------------------+--------------------+
| 5002.378270466019|1979.1666666666667|(5743,[0,693,5730...|
|6434.2785311776925|2930.4030122974386|(5743,[0,4271,572...|
|3140.5424096144716|           3281.25|(5743,[0,190,5729...|
| 4468.345442077481|3888.8888888888887|(5743,[0,344,5733...|
| 7230.338496187889|            8500.0|(5743,[0,98,5733,...|
| 5910.191834199165| 4566.596268161447|(5743,[0,32,5729,...|
|  9358.46106437909|7499.9999241610885|(5743,[0,1,5729,5...|
| 6517.536829576571| 6069.214684692424|(5743,[0,205,5729...|
| 6697.684508174843|            6300.0|(5743,[0,255,5731...|
| 5329.275021577563| 4705.882352941177|(5743,[0,169,5729...|
+------------------+------------------+--------------------+
only showing top 10 rows

RMSE test: 2245.7927752506034

In [50]:
bestModel = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
print("Best model")
print(f"regParam {bestModel._java_obj.getRegParam()}")
print(f"fitIntercept {bestModel._java_obj.getFitIntercept()}")
print(f"elasticNetParam {bestModel._java_obj.getElasticNetParam()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best model
regParam 0.3
fitIntercept True
elasticNetParam 1.0

### Decission Tree

In [58]:
from pyspark.ml.regression import DecisionTreeRegressor

model = DecisionTreeRegressor(featuresCol='features', labelCol='Cena za m2')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
paramGrid = ParamGridBuilder()\
    .addGrid(model.maxDepth, [5, 10, 20])\
    .build()

crossval = CrossValidator(estimator=model,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(predictionCol="prediction", labelCol="Cena za m2",metricName="rmse"),
                          numFolds=5)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(df_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
cv_predictions = cvModel.transform(df_train)
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE train: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE train: 3450.48132731164

In [62]:
cv_predictions = cvModel.transform(df_test)
cv_predictions.select("prediction","Cena za m2","features").show(10)

from pyspark.ml.evaluation import RegressionEvaluator
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE test: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|        prediction|        Cena za m2|            features|
+------------------+------------------+--------------------+
| 8421.293365308975|1979.1666666666667|(5743,[0,693,5730...|
| 7042.564309245756|2930.4030122974386|(5743,[0,4271,572...|
| 8160.175528268287|           3281.25|(5743,[0,190,5729...|
| 8421.293365308975|3888.8888888888887|(5743,[0,344,5733...|
|10555.953775136624|            8500.0|(5743,[0,98,5733,...|
| 8063.340709585578| 4566.596268161447|(5743,[0,32,5729,...|
|11261.780788566393|7499.9999241610885|(5743,[0,1,5729,5...|
| 7998.067112745754| 6069.214684692424|(5743,[0,205,5729...|
|10605.672486384115|            6300.0|(5743,[0,255,5731...|
| 8160.175528268287| 4705.882352941177|(5743,[0,169,5729...|
+------------------+------------------+--------------------+
only showing top 10 rows

RMSE test: 3511.8742302274304

In [63]:
bestModel = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
print("Best model")
print(f"maxDepth {bestModel._java_obj.getMaxDepth()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best model
regParam 20

### Random Forest

In [65]:
from pyspark.ml.regression import RandomForestRegressor

model = RandomForestRegressor(featuresCol='features', labelCol='Cena za m2')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
paramGrid = ParamGridBuilder()\
    .addGrid(model.maxDepth, [20, 30])\
    .build()

crossval = CrossValidator(estimator=model,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(predictionCol="prediction", labelCol="Cena za m2",metricName="rmse"),
                          numFolds=5)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(df_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://ip-172-31-5-10.ec2.internal:8998/sessions/0/statements/67 with error payload: {"msg":"requirement failed: Session isn't active."}


In [61]:
cv_predictions = cvModel.transform(df_train)
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE train: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE train: 3450.48132731164

In [62]:
cv_predictions = cvModel.transform(df_test)
cv_predictions.select("prediction","Cena za m2","features").show(10)

from pyspark.ml.evaluation import RegressionEvaluator
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE test: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|        prediction|        Cena za m2|            features|
+------------------+------------------+--------------------+
| 8421.293365308975|1979.1666666666667|(5743,[0,693,5730...|
| 7042.564309245756|2930.4030122974386|(5743,[0,4271,572...|
| 8160.175528268287|           3281.25|(5743,[0,190,5729...|
| 8421.293365308975|3888.8888888888887|(5743,[0,344,5733...|
|10555.953775136624|            8500.0|(5743,[0,98,5733,...|
| 8063.340709585578| 4566.596268161447|(5743,[0,32,5729,...|
|11261.780788566393|7499.9999241610885|(5743,[0,1,5729,5...|
| 7998.067112745754| 6069.214684692424|(5743,[0,205,5729...|
|10605.672486384115|            6300.0|(5743,[0,255,5731...|
| 8160.175528268287| 4705.882352941177|(5743,[0,169,5729...|
+------------------+------------------+--------------------+
only showing top 10 rows

RMSE test: 3511.8742302274304

In [63]:
bestModel = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
print("Best model")
print(f"maxDepth {bestModel._java_obj.getMaxDepth()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best model
regParam 20

# ML - więcej zmiennych

### Podział na train, test

In [41]:
train, test = df.randomSplit([0.8, 0.2], seed=10)
train.cache()
test.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Cena: float, Cena za m2: double, Szerokosc geograficzna: double, Dlugosc geograficzna: double, Kod pocztowy: string, Miejscowosc: string, Wojewodztwo: string, Powierzchnia calkowita: float, Liczba pokoi: float, Pietro: float, Liczba pieter w budynku: float, Rok budowy: float, Material: string, Typ budynku: string]

In [42]:
print(f"Zbiór train: {train.count()} rekordów")
print(f"Zbiór test: {test.count()} rekordów")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Zbi?r train: 79235 rekord?w
Zbi?r test: 19647 rekord?w

### Przygotowanie train, test do modelu ML

In [43]:
model_cols = ["Cena za m2", "Kod pocztowy", "Powierzchnia calkowita", "Typ budynku", "Material", "Pietro", "Rok budowy"]

train = train.select(model_cols) #.withColumn("Kod pocztowy", df["Kod pocztowy"].substr(0,5))
test = test.select(model_cols) #.withColumn("Kod pocztowy", df["Kod pocztowy"].substr(0,5))
train.withColumn("Rodzaj rynku", f.when(train["Rok budowy"] >= 2021, "pierwotny").when(train["Rok budowy"] < 2021, "wtórny"))
test.withColumn("Rodzaj rynku", f.when(test["Rok budowy"] >= 2021, "pierwotny").when(test["Rok budowy"] < 2021, "wtórny"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Cena za m2: double, Kod pocztowy: string, Powierzchnia calkowita: float, Typ budynku: string, Material: string, Pietro: float, Rok budowy: float, Rodzaj rynku: string]

In [44]:
catCols = [x for (x, dataType) in train.dtypes if dataType == 'string']
numCols = [x for (x, dataType) in train.dtypes if ((dataType != 'string') & (x != "Cena za m2"))]
print(f"Zmienne numeryczne: {numCols}")
print(f"Zmienne kategoryczne: {catCols}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Zmienne numeryczne: ['Powierzchnia calkowita', 'Pietro', 'Rok budowy']
Zmienne kategoryczne: ['Kod pocztowy', 'Typ budynku', 'Material']

In [45]:
train = train.fillna("other", subset=catCols)
test = test.fillna("other", subset=catCols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### string indexer + one hot encoder + vector assembler + (standard scaler)

In [32]:
from pyspark.ml.feature import Imputer, OneHotEncoderEstimator, StringIndexer, VectorAssembler, StandardScaler

imputer = Imputer(inputCols=numCols, outputCols=numCols)

string_indexer = [
    StringIndexer(inputCol=x, outputCol=x + "_StringIndexer", handleInvalid='skip')
    for x in catCols
]

one_hot_encoder = [
    OneHotEncoderEstimator(
        inputCols=[f"{x}_StringIndexer" for x in catCols],
        outputCols=[f"{x}_OneHotEncoder" for x in catCols],
    )
]

assemblerInput = [x for x in numCols]
assemblerInput += [f"{x}_OneHotEncoder" for x in catCols]

vector_assembler = VectorAssembler(
    inputCols=assemblerInput, outputCol="features_2"
)

scaler = StandardScaler(inputCol="features_2", outputCol="features",
                        withStd=True, withMean=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### pipeline

In [33]:
stages = []
stages += [imputer]
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]
stages += [scaler]

pipeline = Pipeline(stages=stages)

model = pipeline.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_train = model.transform(train).select("Cena za m2", "features")
df_test = model.transform(test).select("Cena za m2", "features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Linear Regression

In [35]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(featuresCol='features', labelCol='Cena za m2', maxIter=10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
paramGrid = ParamGridBuilder()\
    .addGrid(model.regParam, [0.3, 0.1, 0.01])\
    .addGrid(model.fitIntercept, [False, True])\
    .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

crossval = CrossValidator(estimator=model,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(predictionCol="prediction", labelCol="Cena za m2",metricName="rmse"),
                          numFolds=5)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(df_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
cv_predictions = cvModel.transform(df_train)
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE train: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE train: 2096.4147530414775

In [38]:
cv_predictions = cvModel.transform(df_test)
cv_predictions.select("prediction","Cena za m2","features").show(10)

from pyspark.ml.evaluation import RegressionEvaluator
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE test: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|        prediction|        Cena za m2|            features|
+------------------+------------------+--------------------+
| 4745.639992149652|1979.1666666666667|(5745,[0,2,695,57...|
| 6229.360997547465|2930.4030122974386|(5745,[0,1,2,4273...|
|3326.2855800165016|           3281.25|(5745,[0,1,2,192,...|
| 4677.668847167013|3888.8888888888887|(5745,[0,2,346,57...|
| 7469.236051934611|            8500.0|(5745,[0,1,2,100,...|
| 5717.739260349899| 4566.596268161447|(5745,[0,2,34,573...|
|  9539.33082446238|7499.9999241610885|(5745,[0,1,2,3,57...|
| 6355.495680811393| 6069.214684692424|(5745,[0,1,2,207,...|
| 6501.068690035423|            6300.0|(5745,[0,2,257,57...|
|6539.1141355688305| 4705.882352941177|(5745,[0,1,2,171,...|
+------------------+------------------+--------------------+
only showing top 10 rows

RMSE test: 2197.8735715557727

In [39]:
bestModel = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
print("Best model")
print(f"regParam {bestModel._java_obj.getRegParam()}")
print(f"fitIntercept {bestModel._java_obj.getFitIntercept()}")
print(f"elasticNetParam {bestModel._java_obj.getElasticNetParam()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best model
regParam 0.3
fitIntercept True
elasticNetParam 1.0

### Isotonic

In [49]:
from pyspark.ml.regression import IsotonicRegression

model = IsotonicRegression(featuresCol='features', labelCol='Cena za m2')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
paramGrid = ParamGridBuilder()\
    .addGrid(model.isotonic, [False, True])\
    .build()

crossval = CrossValidator(estimator=model,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(predictionCol="prediction", labelCol="Cena za m2",metricName="rmse"),
                          numFolds=5)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(df_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
cv_predictions = cvModel.transform(df_train)
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE train: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE train: 4076.5444049434195

In [52]:
cv_predictions = cvModel.transform(df_test)
cv_predictions.select("prediction","Cena za m2","features").show(10)

from pyspark.ml.evaluation import RegressionEvaluator
cv_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE test: {cv_evaluator.evaluate(cv_predictions)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|        prediction|        Cena za m2|            features|
+------------------+------------------+--------------------+
|            8937.5|1979.1666666666667|(5745,[0,2,695,57...|
|  8907.27458586514|2930.4030122974386|(5745,[0,1,2,4273...|
|  8907.27458586514|           3281.25|(5745,[0,1,2,192,...|
|  8907.27458586514|3888.8888888888887|(5745,[0,2,346,57...|
|11532.903882027633|            8500.0|(5745,[0,1,2,100,...|
| 8942.579081486536| 4566.596268161447|(5745,[0,2,34,573...|
|11449.437519889743|7499.9999241610885|(5745,[0,1,2,3,57...|
| 9693.329470711857| 6069.214684692424|(5745,[0,1,2,207,...|
| 9693.329470711857|            6300.0|(5745,[0,2,257,57...|
|  8907.27458586514| 4705.882352941177|(5745,[0,1,2,171,...|
+------------------+------------------+--------------------+
only showing top 10 rows

RMSE test: 4032.7570912569904

In [53]:
bestModel = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
print("Best model")
print(f"isotonic {bestModel._java_obj.getIsotonic()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best model
isotonic False

# Zapis najlepszego modelu

In [ ]:
bestModel = cvModel.bestModel

In [ ]:
bestModel.save("s3://mieszkania2-pl/lrModel")

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

In [ ]:
lrModel = LinearRegressionModel.load("s3://mieszkania2-pl/lrModel")

In [ ]:
lr_predictions = lrModel.transform(df_test)
lr_predictions.select("prediction","Cena za m2","features").show(10)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="Cena za m2",metricName="rmse")
print(f"RMSE test: {lr_evaluator.evaluate(lr_predictions)}")